### Quantum Variational Algorithms for RNA Folding (or any QUBO/Ising Problem) ###

We assume you already have an Ising Hamiltonian qubit_op and offset offset obtained from your QUBO problem (via qp.to_ising()).
1. VQE Approach
Variational Quantum Eigensolver (VQE) uses a parameterized ansatz circuit and classical optimizer to minimize the expected energy of the Hamiltonian.
Key Steps:
Construct an ansatz (parameterized circuit), e.g., RealAmplitudes or EfficientSU2.
Choose an optimizer like COBYLA or SPSA.
Use an estimator primitive (for simulation use StatevectorEstimator).
Run VQE to find minimal eigenvalue (ground state energy).
Code Example (Qiskit 2.1.x Style)

In [8]:
from qiskit_optimization import QuadraticProgram
import numpy as np

# Assume QUBO Q matrix defined here for problem
Q = np.array([[-2, 1], [1, -2]])

# Create QuadraticProgram, then convert to Ising operator
qp = QuadraticProgram()
qp.binary_var('x0')
qp.binary_var('x1')
qp.minimize(quadratic=Q, linear=[0, 0])
qubit_op, offset = qp.to_ising()

# Ansatz
from qiskit.circuit.library import RealAmplitudes
ansatz = RealAmplitudes(num_qubits=qubit_op.num_qubits, reps=2, entanglement='full')

# Primitives and VQE
from qiskit.primitives import StatevectorEstimator
from qiskit_algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import COBYLA  # or SPSA

estimator = StatevectorEstimator()
optimizer = COBYLA(maxiter=200)    # or SPSA(maxiter=200)
vqe_solver = VQE(ansatz=ansatz, optimizer=optimizer, estimator=estimator)

result = vqe_solver.compute_minimum_eigenvalue(operator=qubit_op)
energy = result.eigenvalue.real + offset
print(f"VQE approximate minimal free energy: {energy}")
print(f"Optimal parameters: {result.optimal_parameters}")


/var/folders/1l/g3s1ytv57hv98hl39w041lxm0000gp/T/ipykernel_2342/3771631530.py:16: DeprecationWarning: The class ``qiskit.circuit.library.n_local.real_amplitudes.RealAmplitudes`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. Use the function qiskit.circuit.library.real_amplitudes instead.
  ansatz = RealAmplitudes(num_qubits=qubit_op.num_qubits, reps=2, entanglement='full')


ImportError: cannot import name 'BaseSampler' from 'qiskit.primitives' (/opt/anaconda3/lib/python3.12/site-packages/qiskit/primitives/__init__.py)

### 2. QAOA Approach ###
Quantum Approximate Optimization Algorithm (QAOA) applies alternating unitaries derived from:
Cost Hamiltonian (problem Hamiltonian)
Mixer Hamiltonian (usually, a transverse-field Hamiltonian)
You specify a depth (number of alternations) p and optimize parameters to minimize energy.
Code Example

In [ ]:
from qiskit_optimization import QuadraticProgram
import numpy as np

Q = np.array([[-2, 1], [1, -2]])
qp = QuadraticProgram()
qp.binary_var('x0')
qp.binary_var('x1')
qp.minimize(quadratic=Q, linear=[0, 0])
qubit_op, offset = qp.to_ising()

from qiskit_algorithms.minimum_eigensolvers import QAOA
from qiskit.algorithms.optimizers import SPSA
from qiskit.primitives import StatevectorSimulator  # For simulation

# Setup optimizer and backend
optimizer = SPSA(maxiter=100)
backend = StatevectorSimulator()

qaoa = QAOA(reps=3, optimizer=optimizer, quantum_instance=backend)

# Run QAOA
result = qaoa.compute_minimum_eigenvalue(operator=qubit_op)
energy = result.eigenvalue.real + offset
print(f"QAOA approximate minimal free energy: {energy}")
print(f"Optimal parameters: {result.optimal_point}")


### 3. VQA Variants: Different Ansätze and Optimizers ###
You can experiment with:
Ansätze: RealAmplitudes, EfficientSU2, hardware-native custom ansatz, or manually constructed circuits.
Optimizers: COBYLA, SPSA (good for noisy hardware), L_BFGS_B, Nelder-Mead, or classical optimizers from SciPy.
Example with Different Ansatz and Optimizer

In [ ]:
from qiskit.circuit.library import EfficientSU2
from qiskit.algorithms.optimizers import SPSA

ansatz = EfficientSU2(num_qubits=qubit_op.num_qubits, entanglement='linear', reps=3)
optimizer = SPSA(maxiter=150)
vqe_solver = VQE(ansatz=ansatz, optimizer=optimizer, estimator=StatevectorEstimator())

result = vqe_solver.compute_minimum_eigenvalue(operator=qubit_op)
energy = result.eigenvalue.real + offset
print(f"VQA with EfficientSU2 + SPSA energy: {energy}")
